# import library

In [1]:
import os
import numpy as np
import tensorflow as tf
from data_loader import *
from data_utils import *
from model import *
from const import *

c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\ten

# prepare data

In [2]:
train_data = getBagImage()

Reading csv................
Load bag image...................
Number of bag train data:  700
---------------------------------------------------------------


In [3]:
def one_hot(index, num_classes):
    tmp = np.zeros(num_classes, dtype=np.float32)
    tmp[index] = 1.0
    return tmp

In [4]:
sess = tf.InteractiveSession()
global_step = tf.contrib.framework.get_or_create_global_step()

x, y, mask = input_tensor()

y_color_conv, y_shape_conv, y_opening_conv, y_strap_conv, y_pattern_conv, y_material_conv, y_handle_conv, y_decoration_conv, is_training, keep_prob = multi_label_net(x)

W0811 14:31:48.698483  5840 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0811 14:31:48.698483  5840 deprecation.py:323] From <ipython-input-4-cc918dc67598>:2: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W0811 14:31:48.704754  5840 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\AI\model.py:8: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 14:31:48.708715  5840 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\AI\ops.py:253: T

In [5]:
color_loss, shape_loss, opening_loss, strap_loss, pattern_loss, material_loss, handle_loss, decoration_loss, l2_loss, total_loss = selective_loss(y_color_conv, y_shape_conv, y_opening_conv, y_strap_conv, y_pattern_conv, y_material_conv, y_handle_conv, y_decoration_conv,y, mask)

train_step = train_op(total_loss, global_step)

color_mask = tf.get_collection('color_mask')[0]
shape_mask = tf.get_collection('shape_mask')[0]
opening_mask = tf.get_collection('opening_mask')[0]
strap_mask = tf.get_collection('strap_mask')[0]
pattern_mask = tf.get_collection('pattern_mask')[0]
material_mask = tf.get_collection('material_mask')[0]
handle_mask = tf.get_collection('handle_mask')[0]
decoration_mask = tf.get_collection('decoration_mask')[0]

y_color = tf.get_collection('y_color')[0]
y_shape = tf.get_collection('y_shape')[0]
y_opening = tf.get_collection('y_opening')[0]
y_strap = tf.get_collection('y_strap')[0]
y_pattern = tf.get_collection('y_pattern')[0]
y_material = tf.get_collection('y_material')[0]
y_handle = tf.get_collection('y_handle')[0]
y_decoration = tf.get_collection('y_decoration')[0]

color_correct_prediction = tf.equal(tf.argmax(y_color_conv, 1), tf.argmax(y_color, 1))
shape_correct_prediction = tf.equal(tf.argmax(y_shape_conv, 1), tf.argmax(y_shape, 1))
opening_correct_prediction = tf.equal(tf.argmax(y_opening_conv, 1), tf.argmax(y_opening, 1))
strap_correct_prediction = tf.equal(tf.argmax(y_strap_conv, 1), tf.argmax(y_strap, 1))
pattern_correct_prediction = tf.equal(tf.argmax(y_pattern_conv, 1), tf.argmax(y_pattern, 1))
material_correct_prediction = tf.equal(tf.argmax(y_material_conv, 1), tf.argmax(y_material, 1))
handle_correct_prediction = tf.equal(tf.argmax(y_handle_conv, 1), tf.argmax(y_handle, 1))
decoration_correct_prediction = tf.equal(tf.argmax(y_decoration_conv, 1), tf.argmax(y_decoration, 1))

color_true_pred = tf.reduce_sum(tf.cast(color_correct_prediction, dtype=tf.float32) * color_mask)
shape_true_pred = tf.reduce_sum(tf.cast(shape_correct_prediction, dtype=tf.float32) * shape_mask)
opening_true_pred = tf.reduce_sum(tf.cast(opening_correct_prediction, dtype=tf.float32) * opening_mask)
strap_true_pred = tf.reduce_sum(tf.cast(strap_correct_prediction, dtype=tf.float32) * strap_mask)
pattern_true_pred = tf.reduce_sum(tf.cast(pattern_correct_prediction, dtype=tf.float32) * pattern_mask)
material_true_pred = tf.reduce_sum(tf.cast(material_correct_prediction, dtype=tf.float32) * material_mask)
handle_true_pred = tf.reduce_sum(tf.cast(handle_correct_prediction, dtype=tf.float32) * handle_mask)
decoration_true_pred = tf.reduce_sum(tf.cast(decoration_correct_prediction, dtype=tf.float32) * decoration_mask)

real_train_data = []

# Mask : color -> 0 , shape -> 1, opening -> 2, strap -> 3, pattern -> 4, material -> 5, handle -> 6, decoration -> 7
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][0], 16), 0.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][1], 16), 1.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][2], 16), 2.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][3], 16), 3.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][4], 16), 4.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][5], 16), 5.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][6], 16), 6.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][7], 16), 7.0))

saver = tf.train.Saver()

W0811 14:31:49.471487  5840 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\AI\model.py:95: The name tf.add_to_collection is deprecated. Please use tf.compat.v1.add_to_collection instead.

W0811 14:31:49.483454  5840 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\AI\model.py:123: The name tf.log is deprecated. Please use tf.math.log instead.

W0811 14:31:49.558264  5840 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\AI\model.py:159: The name tf.train.exponential_decay is deprecated. Please use tf.compat.v1.train.exponential_decay instead.

W0811 14:31:49.564266  5840 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\AI\model.py:160: The name tf.train.MomentumOptimizer is deprecated. Please use tf.compat.v1.train.MomentumOptimizer instead.



In [6]:
if not os.path.isfile(SAVE_FOLDER + 'model.ckpt.index'):
    print('Create new model')
    sess.run(tf.global_variables_initializer())
    print('OK')
else:
    print('Restoring existed model')
    saver.restore(sess, SAVE_FOLDER + 'model.ckpt')
    print('OK')

loss_summary_placeholder = tf.placeholder(tf.float32)
tf.summary.scalar('loss', loss_summary_placeholder)
merge_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./summary/")

learning_rate = tf.get_collection('learning_rate')[0]

Create new model
OK


In [ ]:
current_epoch = (int)(global_step.eval() / (len(real_train_data) // BATCH_SIZE))
for epoch in range(current_epoch + 1, NUM_EPOCHS):
    print('Epoch:', str(epoch))
    np.random.shuffle(real_train_data)
    train_img = []
    train_label = []
    train_mask = []

    for i in range(len(real_train_data)):
        train_img.append(real_train_data[i][0])
        train_label.append(real_train_data[i][1])
        train_mask.append(real_train_data[i][2])

    number_batch = len(real_train_data) // BATCH_SIZE

    avg_ttl = []
    avg_rgl = []
    avg_color_loss = []
    avg_shape_loss = []
    avg_opening_loss = []
    avg_strap_loss = []
    avg_pattern_loss = []
    avg_material_loss = []
    avg_handle_loss = []
    avg_decoration_loss = []

    color_nb_true_pred = 0
    shape_nb_true_pred = 0
    opening_nb_true_pred = 0
    strap_nb_true_pred = 0
    pattern_nb_true_pred = 0
    material_nb_true_pred = 0
    handle_nb_true_pred = 0
    decoration_nb_true_pred = 0

    color_nb_train = 0
    shape_nb_train = 0
    opening_nb_train = 0
    strap_nb_train = 0
    pattern_nb_train = 0
    material_nb_train = 0
    handle_nb_train = 0
    decoration_nb_train = 0

    print("Learning rate: %f" % learning_rate.eval())
    for batch in range(number_batch):
        # print('Training on batch {0}/{1}'.format(str(batch + 1), str(number_batch)))
        top = batch * BATCH_SIZE
        bot = min((batch + 1) * BATCH_SIZE, len(real_train_data))
        batch_img = np.asarray(train_img[top:bot])
        batch_label = np.asarray(train_label[top:bot])
        batch_mask = np.asarray(train_mask[top:bot])
        

        for i in range(BATCH_SIZE):
            if batch_mask[i] == 0.0:
                color_nb_train += 1
            else:
                if batch_mask[i] == 1.0:
                    shape_nb_train += 1
                else:
                    if batch_mask[i] == 2.0:
                        opening_nb_train += 1
                    else:
                        if batch_mask[i] == 3.0:
                            strap_nb_train += 1
                        else:
                            if batch_mask[i] == 4.0:
                                pattern_nb_train += 1
                            else:
                                if batch_mask[i] == 5.0:
                                    material_nb_train += 1
                                else:
                                    if batch_mask[i] == 6.0:
                                        handle_nb_train += 1
                                    else:
                                        decoration_nb_train +=1

        batch_img = augmentation(batch_img, 64)

        ttl, colorl, shapel, openingl, strapl, patternl, materiall, handlel, decol, l2l, _ = sess.run([total_loss, color_loss, shape_loss, opening_loss, strap_loss,
                                                                                      pattern_loss, material_loss, handle_loss, decoration_loss, l2_loss, train_step],
                                                                                      feed_dict={x: batch_img, y: batch_label, mask: batch_mask, is_training: True, keep_prob: DROP_OUT_PROB})

        color_nb_true_pred += sess.run(color_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: True,keep_prob: DROP_OUT_PROB})

        shape_nb_true_pred += sess.run(shape_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: True, keep_prob: DROP_OUT_PROB})

        opening_nb_true_pred += sess.run(opening_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                       is_training: True, keep_prob: DROP_OUT_PROB})

        strap_nb_true_pred += sess.run(strap_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: True, keep_prob: DROP_OUT_PROB})

        pattern_nb_true_pred += sess.run(pattern_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                       is_training: True, keep_prob: DROP_OUT_PROB})

        material_nb_true_pred += sess.run(material_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                         is_training: True, keep_prob: DROP_OUT_PROB})

        handle_nb_true_pred += sess.run(handle_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                         is_training: True, keep_prob:DROP_OUT_PROB})

        decoration_nb_true_pred += sess.run(decoration_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                         is_training: True, keep_prob: DROP_OUT_PROB})

        avg_ttl.append(ttl)
        avg_color_loss.append(colorl)
        avg_shape_loss.append(shapel)
        avg_opening_loss.append(openingl)
        avg_strap_loss.append(strapl)
        avg_pattern_loss.append(patternl)
        avg_material_loss.append(materiall)
        avg_handle_loss.append(handlel)
        avg_decoration_loss.append(decol)

        avg_rgl.append(l2l)

    color_train_accuracy = color_nb_true_pred * 1.0 / color_nb_train
    shape_train_accuracy = shape_nb_true_pred * 1.0 / shape_nb_train
    opening_train_accuracy = opening_nb_true_pred * 1.0 / opening_nb_train
    strap_train_accuracy = strap_nb_true_pred * 1.0 / strap_nb_train
    pattern_train_accuracy = pattern_nb_true_pred * 1.0 / pattern_nb_train
    material_train_accuracy = material_nb_true_pred * 1.0 / material_nb_train
    handle_train_accuracy = handle_nb_true_pred * 1.0 / handle_nb_train
    decoration_train_accuracy = decoration_nb_true_pred * 1.0 / decoration_nb_train


    #     print('Avg_ttl: ' + str(avg_ttl))
    #     print('loss_summary_placeholder: ' + str(loss_summary_placeholder))
    #     print('merge_summary: ' + str(merge_summary))
    
    # 밑의 두 줄 error 발생해서 주석처리 해놓았음. TODO!
    
    #summary = sess.run(merge_summary, feed_dict={loss_summary_placeholder: avg_ttl})
    #writer.add_summary(summary, global_step=epoch)

    with open('log.csv', 'w+') as f:
        # epochs, color_train_accuracy, shape_train_accuracy, opening_train_accuracy,
        # avg_color_loss, avg_shape_loss, avg_opening_loss, avg_ttl, avg_rgl
        f.write('{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13},{14}, {15}, {16}\n'.format(current_epoch, color_train_accuracy, shape_train_accuracy, opening_train_accuracy,
                                                                                            strap_train_accuracy, pattern_train_accuracy, material_train_accuracy,
                                                                                            avg_color_loss, avg_shape_loss, avg_opening_loss, avg_strap_loss, avg_pattern_loss,
                                                                                            avg_material_loss, avg_handle_loss, avg_decoration_loss, avg_ttl, avg_rgl))

    print('color task train accuracy: ' + str(color_train_accuracy * 100))
    print('shape task train accuracy: ' + str(shape_train_accuracy * 100))
    print('opening task train accuracy: ' + str(opening_train_accuracy * 100))
    print('strap task train accuracy: ' + str(strap_train_accuracy * 100))
    print('pattern task train accuracy: ' + str(pattern_train_accuracy * 100))
    print('material task train accuracy: ' + str(material_train_accuracy * 100))
    print('handle task train accuracy: ' + str(handle_train_accuracy * 100))
    print('decoration task train accuracy: ' + str(decoration_train_accuracy * 100))

#     print('Total loss: ' + str(avg_ttl) + '. L2-loss: ' + str(avg_rgl))
#     print('Color loss: ' + str(avg_color_loss))
#     print('Shape loss: ' + str(avg_shape_loss))
#     print('opening loss: ' + str(avg_opening_loss))
#     print('strap loss: ' + str(avg_strap_loss))
#     print('pattern loss: ' + str(avg_pattern_loss))
#     print('material loss: ' + str(avg_material_loss))
#     print('handle loss: ' + str(avg_handle_loss))
#     print('decoration loss: ' + str(avg_decoration_loss))

    print('\n')

    saver.save(sess, SAVE_FOLDER + 'model.ckpt')

Epoch: 1
Learning rate: 0.001000
color task train accuracy: 20.285714285714285
shape task train accuracy: 44.714285714285715
opening task train accuracy: 45.285714285714285
strap task train accuracy: 37.42857142857143
pattern task train accuracy: 63.857142857142854
material task train accuracy: 79.28571428571428
handle task train accuracy: 80.28571428571428
decoration task train accuracy: 58.857142857142854


Epoch: 2
Learning rate: 0.001000
color task train accuracy: 30.714285714285715
shape task train accuracy: 45.285714285714285
opening task train accuracy: 49.0
strap task train accuracy: 45.714285714285715
pattern task train accuracy: 51.28571428571429
material task train accuracy: 80.28571428571428
handle task train accuracy: 80.28571428571428
decoration task train accuracy: 60.0


Epoch: 3
Learning rate: 0.001000
color task train accuracy: 34.0
shape task train accuracy: 44.714285714285715
opening task train accuracy: 52.714285714285715
strap task train accuracy: 51.5714285714285